In [69]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import torchvision
import torch 
import neural_tangents as nt

In [17]:
## IMPORT DATASET HERE


mnist_set_train = torchvision.datasets.MNIST('./files/', train=True, download=True,
transform=torchvision.transforms.Compose([
torchvision.transforms.ToTensor()
# torchvision.transforms.Normalize(
# (0.1307,), (0.3081,))
])
)
mnist_set_train = list(mnist_set_train)
X = torch.cat([x[0].reshape(1,-1) for x in mnist_set_train],dim=0).numpy()
Y = torch.cat([torch.tensor(x[1]).reshape(1,-1)*1.0 for x in mnist_set_train],dim=0).numpy()
Y = Y[:,0] 
mX = X #np.load('mnist_x.npy')
mY = Y #np.load('mnist_y.npy')

mnist_set_test = torchvision.datasets.MNIST('./files/', train=False, download=True,
transform=torchvision.transforms.Compose([
torchvision.transforms.ToTensor()
# torchvision.transforms.Normalize(
# (0.1307,), (0.3081,))
])
)
mnist_set_test = list(mnist_set_test)
X = torch.cat([x[0].reshape(1,-1) for x in mnist_set_test],dim=0).numpy()
Y = torch.cat([torch.tensor(x[1]).reshape(1,-1)*1.0 for x in mnist_set_test],dim=0).numpy()
Y = Y[:,0] 
mX_test = X #np.load('mnist_x.npy')
mY_test = Y #np.load('mnist_y.npy')
# Normalize by mean
# mX = mX - mX.mean(axis=0) # Take mean of each feature across all data points (along coloumns),
# mY = mY - mY.mean()
# mY = mY / mY.std()
# mX /= mX.std(axis=0)
# # Whitening covariance
# Sigma = np.cov(mX.T)
# D, V = np.linalg.eig(Sigma)
# D = np.real(D)
# V = np.real(V)
# mX = (V @ np.diag(np.abs(D+1e-10)**(-0.5)) @ V.T) @ mX.T
# mX = mX.T
# # Remove dependent covariates
# Z = np.diagonal(np.cov(mX.T))
# mX = mX[:,Z > 0.95]

In [19]:
mY_test.shape

(10000,)

In [20]:
## GLOBALS
NDATA = mX.shape[0] 
DIM = mX.shape[1] 
BATCH_SIZE = 100 # SET TO NDATA IF FULL BATCH
STEPS_PER_EPOCH = NDATA//BATCH_SIZE

## TRAINING PROCEDURES
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=6000,
  decay_rate=1,
  staircase=False)

lr_schedule_cosine = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate = 0.001,
    first_decay_steps = 6000,
    t_mul=2.0,
    m_mul=1.0,
    alpha=0.0001,
    name=None
)

def get_optimizer():
  # return tf.keras.optimizers.Adam(learning_rate=0.0001)
  return tf.keras.optimizers.Adam(lr_schedule)

def get_callbacks(name):
  return [
    # tfdocs.modeling.EpochDots(),
    # tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=400),
  ]

class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

    Arguments:
        tol: minimum loss
    """

    def __init__(self, tol = 1e-2):
        super().__init__()
        self.tol = tol
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.stopped_epoch = 0

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current,self.tol):
          self.stopped_epoch = epoch
          self.best_weights = self.model.get_weights()
          self.model.stop_training = True
          self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        # if self.stopped_epoch > 0:
        print("Epoch %05d: loss %02f" % (self.stopped_epoch + 1, logs.get("loss")))

def l2(y_true, y_pred):
    return tf.square(y_true - y_pred) 

def compile_and_fit(model, name, x_train, y_train, optimizer=None, max_epochs=6000):
  if optimizer is None:
    optimizer = get_optimizer()
  model.compile(optimizer=optimizer,
                loss=l2,
                run_eagerly=True)

  # model.summary()

  history = model.fit(
    x_train,
    y_train,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs=max_epochs,
    # callbacks=get_callbacks(name), # add for early stopping
    callbacks=EarlyStoppingAtMinLoss(tol=1e-3),
    verbose=1)
    # validation_split = 0.2)
  return history

def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  # plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Training Loss')
  plt.legend()
  plt.grid(True)


In [21]:
## Single run

width = 200

model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(width, activation='relu',input_shape=(DIM,)),
        tf.keras.layers.Dense(width, activation='relu',input_shape=(DIM,)),
        tf.keras.layers.Dense(width, activation='relu',input_shape=(DIM,)),
        tf.keras.layers.Dense(units=1)
      ])

history = compile_and_fit(model,"RELU 500 - MNIST",mX,mY)
# print("\n")
# print("Test loss = ",history.history['val_loss'][-1])

Epoch 1/6000
600/600 [==============================] - 11s 18ms/step - loss: 1.7229
Epoch 2/6000
600/600 [==============================] - 11s 18ms/step - loss: 0.6620
Epoch 3/6000
600/600 [==============================] - 11s 18ms/step - loss: 0.4616
Epoch 4/6000
600/600 [==============================] - 11s 18ms/step - loss: 0.3424
Epoch 5/6000
600/600 [==============================] - 11s 18ms/step - loss: 0.2617
Epoch 6/6000
600/600 [==============================] - 11s 18ms/step - loss: 0.2040
Epoch 7/6000
600/600 [==============================] - 11s 18ms/step - loss: 0.1597
Epoch 8/6000
600/600 [==============================] - 11s 18ms/step - loss: 0.1370
Epoch 9/6000
600/600 [==============================] - 11s 18ms/step - loss: 0.1008
Epoch 10/6000
600/600 [==============================] - 11s 18ms/step - loss: 0.0809
Epoch 11/6000
600/600 [==============================] - 11s 18ms/step - loss: 0.0692
Epoch 12/6000
600/600 [==============================] - 11s 18

In [70]:
def get_ntk_fns():
  # Define a TF-Keras ResNet50 with `O` output logits.
  f = model
  _, params = nt.experimental.get_apply_fn_and_params(f)

  kwargs = dict(
      f=f,
      trace_axes=(),
      vmap_axes=0
  )

  # Different NTK implementations
  jacobian_contraction = nt.experimental.empirical_ntk_fn_tf(
      **kwargs, implementation=nt.NtkImplementation.JACOBIAN_CONTRACTION)
  ntvp = nt.experimental.empirical_ntk_fn_tf(
      **kwargs, implementation=nt.NtkImplementation.NTK_VECTOR_PRODUCTS)
  str_derivatives = nt.experimental.empirical_ntk_fn_tf(
      **kwargs, implementation=nt.NtkImplementation.STRUCTURED_DERIVATIVES)
  auto = nt.experimental.empirical_ntk_fn_tf(
      **kwargs, implementation=nt.NtkImplementation.AUTO)
  
  return params, (jacobian_contraction, ntvp, str_derivatives, auto)

In [71]:
params, (ntk_fn_jacobian_contraction, ntk_fn_ntvp, ntk_fn_str_derivatives, ntk_fn_auto) = get_ntk_fns()

C:\Users\s4531973\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neural_tangents\experimental\empirical_tf\empirical.py:232: UserWarning: This function is an early proof-of-concept.
  warnings.warn('This function is an early proof-of-concept.')


In [87]:
mX.shape


(60000, 784)

In [89]:
k_1 = ntk_fn_ntvp(mX, mX, params)
print(k_1)

ResourceExhaustedError: Out of memory while trying to allocate 8640192000000 bytes. [Op:__inference_converted_fun_tf_14064494]